In [39]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import matplotlib.pyplot as plt
import seaborn as sns


In [40]:
data = pd.read_csv('raw_data/VMcn4Vml.csv')

# Display the first few rows of the dataset
print(data.head())

  IndicatorCode                                          Indicator ValueType  \
0       SDGPM25  Concentrations of fine particulate matter (PM2.5)      text   
1       SDGPM25  Concentrations of fine particulate matter (PM2.5)      text   
2       SDGPM25  Concentrations of fine particulate matter (PM2.5)      text   
3       SDGPM25  Concentrations of fine particulate matter (PM2.5)      text   
4       SDGPM25  Concentrations of fine particulate matter (PM2.5)      text   

  ParentLocationCode ParentLocation Location type SpatialDimValueCode  \
0                AFR         Africa       Country                 KEN   
1                AMR       Americas       Country                 TTO   
2                EUR         Europe       Country                 GBR   
3                AMR       Americas       Country                 GRD   
4                AMR       Americas       Country                 BRA   

                                            Location Period type  Period  ...  \

In [41]:
# Keep only relevant columns
columns_to_keep = ['Location', 'Period', 'FactValueNumericLow', 'FactValueNumericHigh', 'Value']
data_cleaned = data[columns_to_keep]

# Check the cleaned dataset
print(data_cleaned.head())



                                            Location  Period  \
0                                              Kenya    2019   
1                                Trinidad and Tobago    2019   
2  United Kingdom of Great Britain and Northern I...    2019   
3                                            Grenada    2019   
4                                             Brazil    2019   

   FactValueNumericLow  FactValueNumericHigh               Value  
0                 6.29                 13.74  10.01 [6.29-13.74]  
1                 7.44                 12.55  10.02 [7.44-12.55]  
2                 9.73                 10.39  10.06 [9.73-10.39]  
3                 7.07                 13.20  10.08 [7.07-13.20]  
4                 8.23                 12.46  10.09 [8.23-12.46]  


In [42]:
# Extract the average PM2.5 value from the 'Value' column
data_cleaned['PM2.5_Avg'] = data_cleaned['Value'].str.extract(r'(\d+\.\d+)').astype(float)

# Drop the old 'Value' column
data_cleaned.drop(columns=['Value'], inplace=True) 

# Check the result
print(data_cleaned.head())

                                            Location  Period  \
0                                              Kenya    2019   
1                                Trinidad and Tobago    2019   
2  United Kingdom of Great Britain and Northern I...    2019   
3                                            Grenada    2019   
4                                             Brazil    2019   

   FactValueNumericLow  FactValueNumericHigh  PM2.5_Avg  
0                 6.29                 13.74      10.01  
1                 7.44                 12.55      10.02  
2                 9.73                 10.39      10.06  
3                 7.07                 13.20      10.08  
4                 8.23                 12.46      10.09  


C:\Users\Erik Dahlin\AppData\Local\Temp\ipykernel_26772\3850545411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['PM2.5_Avg'] = data_cleaned['Value'].str.extract(r'(\d+\.\d+)').astype(float)
C:\Users\Erik Dahlin\AppData\Local\Temp\ipykernel_26772\3850545411.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.drop(columns=['Value'], inplace=True)


In [43]:
# Check for missing values
print(data_cleaned.isnull().sum())

# Option 1: Drop rows with missing values
data_cleaned = data_cleaned.dropna()
data_cleaned = data_cleaned.drop_duplicates(subset=['Location', 'Period'])

data_cleaned.columns = data_cleaned.columns.astype(str)
# Set the 'Location' column as the index
data_cleaned.set_index('Location', inplace=True)

# Calculate the average change in PM2.5 over the years for each country
data_cleaned['Average Change'] = data_cleaned.loc[:, '2018':'2019'].diff(axis=1).mean(axis=1)
# Option 2: Impute missing values (e.g., using the mean)
# data_cleaned['FactValueNumericLow'].fillna(data_cleaned['FactValueNumericLow'].mean(), inplace=True)
# data_cleaned['FactValueNumericHigh'].fillna(data_cleaned['FactValueNumericHigh'].mean(), inplace=True)

Location                0
Period                  0
FactValueNumericLow     0
FactValueNumericHigh    0
PM2.5_Avg               0
dtype: int64


KeyError: '2018'

In [18]:
# Standardize the PM2.5 values
scaler = StandardScaler()
data_cleaned[['FactValueNumericLow', 'FactValueNumericHigh', 'PM2.5_Avg']] = scaler.fit_transform(
    data_cleaned[['FactValueNumericLow', 'FactValueNumericHigh', 'PM2.5_Avg']])

# Check the standardized data
print(data_cleaned.head())


                                            Location  Period  \
0                                              Kenya    2019   
1                                Trinidad and Tobago    2019   
2  United Kingdom of Great Britain and Northern I...    2019   
3                                            Grenada    2019   
4                                             Brazil    2019   

   FactValueNumericLow  FactValueNumericHigh  PM2.5_Avg  
0            -0.836418             -0.794445  -0.835705  
1            -0.717380             -0.848285  -0.834944  
2            -0.480340             -0.946013  -0.831901  
3            -0.755679             -0.818877  -0.830379  
4            -0.635607             -0.852357  -0.829618  


In [19]:
# Save the preprocessed data to a new CSV file
data_cleaned.to_csv('preprocessed_data/preprocessed_pm25_data.csv', index=False)
